In [2]:
import pandas as pd
from pandas import read_csv
from pandas import datetime
import numpy as np
from numpy import array
from numpy import newaxis
import math
from scipy import stats
from subprocess import check_output

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import  train_test_split
import time #helper libraries
import matplotlib.pyplot as plt

Analysis on Singapore Fuel Oil 380 cst (Platts)

In [98]:
def parser(x):
     return datetime.strptime(x, '%m/%d/%Y')
 
df = pd.read_csv('big_df_adj.csv', header=0, keep_default_na=False, index_col = 0, parse_dates=[0], date_parser=parser,  low_memory=False)


In [100]:
df_t = df['CME_CL1_FWs']


In [29]:
def get_technical_indicators(dataset):
    # Create 7 and 21 days Moving Average
    dataset['ma7'] = dataset.rolling(7).mean()
    dataset['ma21'] = dataset.rolling(21).mean()

    # Create MACD
    dataset['26ema'] = pd.ewma(dataset['CME_SES'], span=26)
    dataset['12ema'] = pd.ewma(dataset['CME_SES'], span=12)
    dataset['MACD'] = (dataset['12ema']-dataset['26ema'])
# Create Bollinger Bands
    dataset['20sd'] = pd.stats.moments.rolling_std(dataset['price'],20)
    dataset['upper_band'] = dataset['ma21'] + (dataset['20sd']*2)
    dataset['lower_band'] = dataset['ma21'] - (dataset['20sd']*2)
    
    # Create Exponential moving average
    dataset['ema'] = dataset['price'].ewm(com=0.5).mean()
    
    # Create Momentum
    dataset['momentum'] = dataset['price']-1
    
    return dataset

In [91]:
df_t['ma7'] = df_t.rolling(7).mean()


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
df_o['ma21'] = df_o['CME_CL1_FWs'].rolling(21).mean()


KeyError: 'CME_CL1_FWs'

In [65]:
df_o['20sd'] = pd.stats.moments.rolling_std(df_o['CME_SES'],20)


AttributeError: module 'pandas' has no attribute 'stats'

,CME_SED,CME_SES
0,2019-05-01,424.371
1,2019-04-30,424.371
2,2019-04-29,424.247
3,2019-04-26,424.298
4,2019-04-24,425.273
5,2019-04-23,423.829
6,2019-04-22,422.835
7,2019-04-18,419.655
8,2019-04-17,419.629
9,2019-04-16,418.610


In [96]:
dataset = pd.read_csv('big_df_adj.csv', header=0, keep_default_na=False, usecols=[19,20], parse_dates=[0], date_parser=parser,  low_memory=False)
df= dataset.fillna(0)
df_z = np.abs(stats.zscore(df_t['CME_CL1_FWs']))
threshhold = 3
print(np.where(df_z>threshhold))
dataset_ex_df = df[(df_z < threshhold)]

ValueError: time data '1029433' does not match format '%m/%d/%Y'

In [94]:
data_FT = df_t
close_fft = np.fft.fft(np.asarray(data_FT['CME_CL1_FWs'].tolist()))
fft_df = pd.DataFrame({'fft':close_fft})
fft_df['absolute'] = fft_df['fft'].apply(lambda x: np.abs(x))
fft_df['angle'] = fft_df['fft'].apply(lambda x: np.angle(x))
plt.figure(figsize=(14, 7), dpi=100)
fft_list = np.asarray(fft_df['fft'].tolist())
for num_ in [3, 6, 9, 100]:
    fft_list_m10= np.copy(fft_list); fft_list_m10[num_:-num_]=0
    plt.plot(np.fft.ifft(fft_list_m10), label='Fourier transform with {} components'.format(num_))
plt.plot(data_FT['CME_CL1_FWs'],  label='Real')
plt.xlabel('Days')
plt.ylabel('USD')
plt.title('Figure 3: Singapore Fuel Oil 380 cst (Platts) & Fourier transforms')
plt.legend()
plt.show()

KeyError: 'CME_CL1_FWs'

ARIMA - Test MSE: 49.587
                    ARIMA Model Results                              
==============================================================================
Dep. Variable:              D.CME_SES   No. Observations:                 1077
Model:                 ARIMA(5, 1, 0)   Log Likelihood               -3423.963
Method:                       css-mle   S.D. of innovations              5.814
Date:                Sat, 11 May 2019   AIC                           6861.925
Time:                        17:39:35   BIC                           6896.799
Sample:                             1   HQIC                          6875.132
                                                                              
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0669      0.191     -0.351      0.726      -0.440       0.307
ar.L1.D.CME_SES     0.0239      0.030      0.786      0.432      -0.036       0.084
ar.L2.D.CME_SES     0.0063      0.030      0.206      0.837      -0.053       0.066
ar.L3.D.CME_SES    -0.0023      0.030     -0.074      0.941      -0.062       0.057
ar.L4.D.CME_SES     0.0139      0.030      0.458      0.647      -0.046       0.074
ar.L5.D.CME_SES     0.0287      0.032      0.901      0.368      -0.034       0.091
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.9237           -0.0000j            1.9237           -0.0000
AR.2            0.5623           -1.9161j            1.9969           -0.2046
AR.3            0.5623           +1.9161j            1.9969            0.2046
AR.4           -1.7669           -1.1919j            2.1313           -0.4055
AR.5           -1.7669           +1.1919j            2.1313            0.4055
-----------------------------------------------------------------------------

In [1]:

regressor = xgb.XGBRegressor(gamma=0.0,n_estimators=150,base_score=0.7,colsample_bytree=1,learning_rate=0.05)
xgbModel = regressor.fit(X_train_FI,y_train_FI, eval_set = [(X_train_FI, y_train_FI), (X_test_FI, y_test_FI)], verbose=False)
fig = plt.figure(figsize=(8,8))
plt.xticks(rotation='vertical')
plt.bar([i for i in range(len(xgbModel.feature_importances_))], xgbModel.feature_importances_.tolist(), tick_label=X_test_FI.columns)
plt.title('Figure 6: Feature importance of the technical indicators.')
plt.show()

NameError: name 'xgb' is not defined